In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from datasets import load_dataset, concatenate_datasets
from sklearn.metrics import classification_report

In [2]:
hiner = load_dataset('cfilt/HiNER-collapsed')
train_data = hiner['train']
val_data = hiner['validation']
test_data = hiner['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset hi_ner_collapsed_config downloaded and prepared to /root/.cache/huggingface/datasets/cfilt___hi_ner_collapsed_config/HiNER-Collapsed/0.0.2/fa4c99b4cefed1144a9c7da5e3d85737ead950f0e8db723d1bf108fc7613b493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
combined_data = concatenate_datasets([train_data, val_data])

In [4]:
tokens = combined_data['tokens']
ner_tags = combined_data['ner_tags']

In [5]:
all_tokens = [token for sublist in tokens for token in sublist]
all_tags = [tag for sublist in ner_tags for tag in sublist]
token_vocab = sorted(set(all_tokens))
tag_vocab = sorted(set(all_tags))
# tag_vocab.append(int(7))

In [6]:
token_to_id = {token: i for i, token in enumerate(token_vocab)}
tag_to_id = {tag: i for i, tag in enumerate(tag_vocab)}

In [7]:
# Preprocess the training data
train_tokens = train_data['tokens']
train_tags = train_data['ner_tags']
train_tokens_ids = [[token_to_id[token] for token in sublist] for sublist in train_tokens]
train_tags_ids = [[tag_to_id[tag] for tag in sublist] for sublist in train_tags]
max_length = max(len(seq) for seq in train_tokens_ids)  # Calculate the maximum length
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=max_length, padding='post')
train_tags_ids = pad_sequences(train_tags_ids, maxlen=max_length, padding='post')
train_tags_ids = to_categorical(train_tags_ids, num_classes=len(tag_vocab))

In [8]:
# Preprocess the validation data
val_tokens = val_data['tokens']
val_tags = val_data['ner_tags']
val_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 6)) for token in sublist] for sublist in val_tokens]
val_tags_ids = [[tag_to_id.get(tag, tag_to_id.get('<unk>', 6)) for tag in sublist] for sublist in val_tags]
val_tokens_ids = pad_sequences(val_tokens_ids, maxlen=max_length, padding='post')
val_tags_ids = pad_sequences(val_tags_ids, maxlen=max_length, padding='post')
val_tags_ids = to_categorical(val_tags_ids, num_classes=len(tag_vocab))

In [9]:
# Preprocess the test data
test_tokens = test_data['tokens']
test_tags = test_data['ner_tags']
test_tokens_ids = [[token_to_id.get(token, token_to_id.get('<unk>', 6)) for token in sublist] for sublist in test_tokens]
test_tags_ids = [[tag_to_id.get(tag, token_to_id.get('<unk>', 6)) for tag in sublist] for sublist in test_tags]
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=max_length, padding='post')
test_tags_ids = pad_sequences(test_tags_ids, maxlen=max_length, padding='post')
test_tags_ids = to_categorical(test_tags_ids, num_classes=len(tag_vocab))

In [10]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=len(token_vocab), output_dim=64),
    layers.Bidirectional(layers.GRU(128, return_sequences=True)),
    layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(tag_vocab), activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [12]:
model.fit(train_tokens_ids, train_tags_ids, validation_data=(val_tokens_ids, val_tags_ids), epochs=5, batch_size=128)

Epoch 1/5
593/593 [==============================] - 593s 985ms/step - loss: 0.0596 - accuracy: 0.9858 - precision: 0.9923 - recall: 0.9723 - val_loss: 0.0127 - val_accuracy: 0.9962 - val_precision: 0.9973 - val_recall: 0.9955
Epoch 2/5
593/593 [==============================] - 595s 1s/step - loss: 0.0084 - accuracy: 0.9975 - precision: 0.9980 - recall: 0.9971 - val_loss: 0.0093 - val_accuracy: 0.9973 - val_precision: 0.9976 - val_recall: 0.9970
Epoch 3/5
593/593 [==============================] - 575s 970ms/step - loss: 0.0055 - accuracy: 0.9984 - precision: 0.9985 - recall: 0.9982 - val_loss: 0.0092 - val_accuracy: 0.9973 - val_precision: 0.9976 - val_recall: 0.9971
Epoch 4/5
593/593 [==============================] - 583s 983ms/step - loss: 0.0043 - accuracy: 0.9987 - precision: 0.9988 - recall: 0.9986 - val_loss: 0.0096 - val_accuracy: 0.9973 - val_precision: 0.9975 - val_recall: 0.9972
Epoch 5/5
593/593 [==============================] - 591s 996ms/step - loss: 0.0035 - accuracy:

In [13]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_tokens_ids, test_tags_ids)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)
print('Test Precision:', test_precision)
print('Test Recall:',test_recall)

677/677 [==============================] - 64s 95ms/step - loss: 0.0113 - accuracy: 0.9972 - precision: 0.9973 - recall: 0.9971
Test Loss: 0.01134392898529768
Test Accuracy: 0.9971664547920227
Test Precision: 0.9973427057266235
Test Recall: 0.9970614910125732


In [14]:
model.save('NLP_NER_BiGRU_Collapsed.h5')

In [15]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions on test data
predictions = model.predict(test_tokens_ids)

# Convert predictions from categorical to label encoded
pred_tags_ids = np.argmax(predictions, axis=-1)

# Convert test tags from one-hot encoded to label encoded
true_tags_ids = np.argmax(test_tags_ids, axis=-1)

# Flatten both arrays
pred_tags_ids_flat = pred_tags_ids.flatten()
true_tags_ids_flat = true_tags_ids.flatten()

# Use inverse_transform to map predicted and true labels back to their original form.
# You can skip this step if you want the classification report for the encoded labels.
pred_tags = [tag_vocab[i] for i in pred_tags_ids_flat]
true_tags = [tag_vocab[i] for i in true_tags_ids_flat]

# Generate classification report
report = classification_report(true_tags, pred_tags, output_dict=True)

report = classification_report(true_tags, pred_tags)
print(report)

677/677 [==============================] - 66s 95ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   3495879
           1       0.83      0.77      0.80      5351
           2       0.87      0.80      0.84      7495
           3       0.84      0.75      0.79      4731
           4       0.79      0.75      0.77      3849
           5       0.90      0.82      0.86      5488
           6       0.99      0.99      0.99    375467

    accuracy                           1.00   3898260
   macro avg       0.89      0.84      0.86   3898260
weighted avg       1.00      1.00      1.00   3898260

